<a href="https://colab.research.google.com/github/abhilasha-kumar/Connector/blob/master/search-models/search_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning git repository & load libraries



In [ ]:
!git clone https://github.com/abhilasha-kumar/Connector.git
!pip install pybind11
!pip install graph-walker # fast random walk implementation from https://github.com/kerighan/graph-walker

# Importing embeddings, vocabulary, & functions file
We load the embeddings, vocabulary, and all the search functions. The functions have been predefined and are stored in the search-models subdirectory in the github repository, so we directly load them here.

In [ ]:
# import glove embeddings
import pandas as pd
import json
import numpy as np
%cd /content/Connector/connector-cogsci2021/data
representations = {}
representations['glove'] = pd.read_csv("glove_embeddings.csv").transpose().values
representations['swow'] = pd.read_csv("swow_embeddings.csv").transpose().values
vocab = pd.read_csv("vocab.csv").rename(columns={"Word": "vocab_word"})
print(f"embeddings are shaped:", representations['glove'].shape)
print(f"vocab is {len(vocab)} words")
with open('boards.json', 'r') as json_file:
    boards = json.load(json_file)

## import empirical clues (cleaned)
expdata = pd.read_csv("final_board_clues_all.csv", encoding= 'unicode_escape')

## need to get similarity matrix of these words in this order to work with
target_df = pd.read_csv("connector_wordpairs_boards.csv")
target_df["wordpair"]= target_df["Word1"]+ "-"+target_df["Word2"]
print(target_df.head())

%cd /content/Connector/search-models
import search_funcs

# Running through full dataset

Having verified the functions, we will now run these functions on the full behavioral dataset. We will need to explore some parameters for this. 



We have 2 variables:
1.   Candidates (full/subset)
2.   Pragmatics (with/without)

We have the following search models that generate candidates:
1.   Union (RW)
2.   Intersection (RW)
3.   Predication (spreading activation)

We have the following models that can be run with/without candidates and with/without pragmatics:
1.   Speaker models
    - Target+Board
    - Pragmatic speaker
2.  Guesser models 
  - Literal Guesser
  - Pragmatic Guesser







### Step 1: On full vocab

In [ ]:
## create boards and merge with expdata
combined_boards_df = pd.DataFrame(columns=['Experiment', 'Board','boardwords'])
combined_boards_df["Experiment"]  = ["E1"] * 10 + ["E2"] * 10
combined_boards_df["Board"] = ["TrialList" + str(i) for i in range(1,11)] * 2
combined_boards_df["boardnames"] = (['e1_board' + str(i) + '_words' for i in range(1,11)] 
                                  + ['e2_board' + str(i) + '_words' for i in range(1,11)])
combined_boards_df["boardwords"] = [boards[n] for n in combined_boards_df["boardnames"]]

expdata_new = pd.merge(expdata,combined_boards_df,on=['Board', 'Experiment'],how='left')
expdata_new["wordpair"] = expdata_new["Word1"] + "-" + expdata_new["Word2"]
board_combos = {board_name : search_funcs.RSA.compute_board_combos(board_name, boards) for board_name in boards.keys()}

#### Non - RSA method


In [ ]:
board_optimal_params = {
    'swow' : (23.488850322875496, 1), # -13204
    'glove' : (20.952928531665275, 1), # -15774.814774380024)
    'bert-sum' : (19.983835225540847, 0.787924454045298),
}

In [ ]:
cluescoredf = search_funcs.nonRSA.speaker_targetboard_cluescores(['swow', 'glove'], board_optimal_params, board_combos, boards, list(vocab.vocab_word), vocab, representations, target_df, expdata_new)
cluescoredf.head()

#### Models with RSA

In [ ]:
rsa_optimal_params = {
    'swow' : (25.1522030761838, 0.03863169001849234),
    'glove' : (22.336514544537227, 0.039),
    'bert-sum' : (29.709602301411962, 0.031659060110267576), #-17533
}

In [ ]:
pragmaticspeakerdf = search_funcs.RSA.get_speaker_df(representations, combined_boards_df,rsa_optimal_params, list(vocab.vocab_word), vocab, expdata_new, board_combos, target_df, boards)
pragmaticspeakerdf.head()

## Step 2: Candidate generation (Union & Intersection)

In [ ]:
## here we generate candidates for each of our wordpairs: stored in target_df

candidates_df = pd.DataFrame()

# keep n_walks fixed to a large number
n_walks = 1000
n_steps = 50

## should probably try a range of values for threshold 

for modelname in ['glove']:
  for threshold in np.arange(0.2, 0.3, 0.1):
    print(f"threshold is {threshold}")
    # sim_matrix = search_funcs.search.create_similarity_matrix(representations[modelname])
    # Graph = search_funcs.search.create_graph(sim_matrix, threshold)
    print(f"graph has been created")
    for index, row in target_df.iterrows():
      w1 = row["Word1"]
      w2 = row["Word2"]
      print(f"for {w1} and {w2}")
      union_df, int_df = search_funcs.search.union_intersection(w1,w2, n_steps, n_walks, vocab, Graph)
      print(f"union/int calculation complete!")
      
      union_df["Word1"] = w1
      union_df["Word2"] = w2
      union_df["representation"] = modelname
      union_df["threshold"] = threshold
      union_df["n_walks"] = n_walks
      union_df["type"] = "union"
      

      int_df["Word1"] = w1
      int_df["Word2"] = w2
      int_df["representation"] = modelname
      int_df["threshold"] = threshold
      int_df["n_walks"] = n_walks
      int_df["type"] = "intersection"

      overall_df = pd.concat([union_df, int_df])

      candidates_df = pd.concat([candidates_df, overall_df])

      

# Writing files to CSV

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
parentfolder = "/content/drive/My Drive/search-models/"
pragmaticspeakerdf.to_csv(parentfolder+'pragmaticspeakerdf.csv')